In [1]:
# %%configure
# { "conf": {"spark.jars.packages": 'spark.jars.packages, org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0' }}

In [2]:
# import findspark
# findspark.init("/opt/spark")

In [3]:
from pyspark.sql import SparkSession
import datetime
import json
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2'
# org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2

In [4]:
# spark = SparkSession.builder.appName("test").getOrCreate()

In [5]:
spark = SparkSession.builder.appName("test").config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0').getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a013733a-5fac-4931-983c-7e0cbd60f588;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.goo

In [6]:
#config
#s3
# s3_path = "s3://emr-kafka-poc/"
output_file = "simple_cluster_consumer.csv"
#spark
spark_app_name = "kafka_poc"
# kafka
username = "devqa"
password = "p@ssw0rd"
msk_bootstrap = "b-3.devqa-msk.4nvszg.c4.kafka.eu-west-1.amazonaws.com:9096,b-2.devqa-msk.4nvszg.c4.kafka.eu-west-1.amazonaws.com:9096,b-1.devqa-msk.4nvszg.c4.kafka.eu-west-1.amazonaws.com:9096"
topics = 'spark_testing'  # for testing, comma separate string for multiple topics
kafka_group_id = "spark_testing"

In [7]:
offset_ts = datetime.datetime(2022,1,12).timestamp()
offset_string = {
    topics : {
        "0" : offset_ts,
        "1" : offset_ts
    }
}

offset_string = json.dumps(offset_string)

options = {
    "kafka.bootstrap.servers" : msk_bootstrap,
    "subscribe" : topics,
    # "startingOffsetsByTimestamp" : offset_string,
    "startingOffsets" : "earliest",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.scram.ScramLoginModule required username="{username}" password="{password}";',
    "kafka.sasl.mechanism": "SCRAM-SHA-512",
    "kafka.security.protocol" : "SASL_SSL",
    "group.id": kafka_group_id
}


df = spark \
    .read \
    .format("kafka") \
    .options(**options) \
    .load()
# df.write.csv(f"{s3_path}{output_file}")
print(f"count of the records is {df.count()}")

count of the records is 1


In [8]:
import sys, os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import Row
from subprocess import check_output


myDF = spark.createDataFrame(
    [
        Row(col0=0, col1=1, col2=2),
        Row(col0=3, col1=1, col2=5),
        Row(col0=6, col1=2, col2=8),
    ]
)



In [9]:
myGDF = myDF.select("*").groupBy("col1")
myDF.createOrReplaceTempView("mydf_as_sqltable")


In [10]:
print(myDF.collect())
# myGDF.sum().show()

# spark.stop()
# quit()

[Row(col0=0, col1=1, col2=2), Row(col0=3, col1=1, col2=5), Row(col0=6, col1=2, col2=8)]
